In [ ]:
# Get rid of annoying tf warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pickle
import numpy as np
np.set_printoptions(suppress=True)

import bayesflow as beef
import tensorflow as tf

import sys
sys.path.append("../")
from experiment import ModelComparisonExperiment

In [ ]:
# gpu setting and checking
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
experiment = ModelComparisonExperiment()

# Training

In [ ]:
with open('../data/training_data.pkl', 'rb') as f:
    training_data = pickle.load(f)
with open('../data/training_validation_data.pkl', 'rb') as f:
    training_validation_data = pickle.load(f)

In [ ]:
history = experiment.run(
    training_data=training_data,
    validation_data=training_validation_data
    )

In [ ]:
f = beef.diagnostics.plot_losses(history)

## Validation

In [ ]:
with open('../data/validation_data.pkl', 'rb') as f:
    validation_data = pickle.load(f)

In [ ]:
configurator = beef.configuration.DefaultModelComparisonConfigurator(4)
validation_data_configured = configurator(validation_data)

In [ ]:
with tf.device('/cpu:0'):
    out = experiment.amortizer.posterior_probs(validation_data_configured)

In [ ]:
cal_curves = beef.diagnostics.plot_calibration_curves(
    true_models=validation_data_configured['model_indices'],
    pred_models=out,
    model_names=['Random walk',
                 'Mixture random walk',
                 'Levy flight',
                 'Regime switching'],
    fig_size=(14, 3)
)

In [ ]:
cal_curves.savefig('../plots/calibration_curves.pdf', dpi=300)

In [ ]:
confusion_matrix = beef.diagnostics.plot_confusion_matrix(
    validation_data_configured['model_indices'],
    out,
    model_names=['RW', 'MRW', 'LF', 'RS']
)

In [ ]:
confusion_matrix.savefig('../plots/confusion_matrix.pdf', dpi=300)